# Planet — Corporate Hybrid Capacity Forecast **v11**
## Hierarchical Forecasting (Vertical → Department) + Dynamic Level Adjustment + Risk-based Staffing + DOW Daily Plan + Real Productivity FTE

**Generated:** 2026-02-14 (Europe/Madrid)

### Executive intent
Increase planning accuracy (especially **Payments** and **Hospitality**) by shifting the signal to a more stable layer:
1) Forecast **monthly volume at Vertical** level  
2) Allocate down to **Department** using **rolling 3-month shares** within each vertical  
3) Apply **dynamic level adjustment** to reduce systematic bias

### What’s new vs v10
- **Hierarchical monthly forecast** (Vertical → Dept)
- **Dynamic level adjustment** (per dept forecast)
- **ARIMA governance** (rejects non-converged fits) + **ARIMA disabled for critical verticals**
- Daily plan remains **DOW-profile based** without changing monthly totals

---


## 1) Setup (interactive BASE_DIR)

In [1]:
import os
from pathlib import Path
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

BASE_DIR = filedialog.askdirectory(title="Select BASE_DIR (CAPACITY folder)")
if not BASE_DIR:
    raise SystemExit("No folder selected. Execution stopped.")

BASE_DIR = str(Path(BASE_DIR).expanduser().resolve())
INPUT_DIR  = str(Path(BASE_DIR) / "input_model")
OUTPUT_DIR = str(Path(BASE_DIR) / "outputs")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

INCOMING_SOURCE_PATH = str(Path(INPUT_DIR) / "Incoming_new.xlsx")
INCOMING_SHEET = "Main"

DEPT_MAP_PATH = str(Path(INPUT_DIR) / "department.xlsx")
DEPT_MAP_SHEET = "map"

PRODUCTIVITY_PATH = str(Path(INPUT_DIR) / "productivity_agents.xlsx")
CASE_REASON_PATH = str(Path(INPUT_DIR) / "case_reason.xlsx")

OUTPUT_XLSX = str(Path(OUTPUT_DIR) / "capacity_forecast_hybrid_v11.xlsx")

# Horizons
H_MONTHS = 12
DAILY_HORIZON_DAYS = 90

SUPPORTED_LANGS = ["Spanish","English","Portuguese","French","German","Italian"]

# Prediction intervals
ENABLE_MONTHLY_PI = True
PI_ALPHA = 0.05  # 95% PI

# Backtesting config (monthly)
ENABLE_ACCURACY_TABLE = True
ACCURACY_BACKTEST_MONTHS = 9
ACCURACY_MIN_TRAIN_MONTHS = 12
ACCURACY_HORIZON_MONTHS = 1
ACCURACY_MAX_SPLITS = 9

# Stability thresholds
STAB = {
    "low_data_min_months": 18,
    "broken_z_thresh": 2.5,
    "volatility_cv_thresh": 0.65,
    "zero_share_thresh": 0.25,
}

# Critical verticals
CRITICAL_VERTICALS = {"Payments","Hospitality"}

# Staffing risk policy
RISK_POLICY = {
    "critical_low_acc_use_p95": 75.0,
    "critical_mid_acc_blend": 88.0,
    "volatility_uplift_pct": 10.0,
    "lowdata_uplift_pct": 15.0,
    "broken_uplift_pct": 20.0,
}

# Level adjustment (bias reduction)
LEVEL_ADJ = {
    "enabled": True,
    "lookback_months": 3,
    "clip_min": 0.70,
    "clip_max": 1.30,
}

# DOW profile
DOW_LOOKBACK_DAYS = 180
DOW_MIN_OBS = 30

required_files = [INCOMING_SOURCE_PATH, DEPT_MAP_PATH, PRODUCTIVITY_PATH, CASE_REASON_PATH]
missing = [p for p in required_files if not Path(p).exists()]
if missing:
    raise FileNotFoundError("Missing required input files:\n- " + "\n- ".join(missing) + f"\n\nSelected BASE_DIR:\n{BASE_DIR}")

print("✅ Setup OK")
print("BASE_DIR:", BASE_DIR)
print("OUTPUT_XLSX:", OUTPUT_XLSX)


✅ Setup OK
BASE_DIR: C:\Projects\Capacity_forecast_2026
OUTPUT_XLSX: C:\Projects\Capacity_forecast_2026\outputs\capacity_forecast_hybrid_v11.xlsx


## 2) Imports

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import math
from typing import Optional, Dict, Tuple
from datetime import date, timedelta

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

try:
    from prophet import Prophet
except Exception:
    try:
        from fbprophet import Prophet
    except Exception:
        Prophet = None

print("Imports loaded. Prophet:", bool(Prophet))


Imports loaded. Prophet: True


## 3) Core functions (v11)

In [3]:
# ============================================================
# CORE FUNCTIONS — v11
# ============================================================

def smape(y_true, y_pred) -> float:
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom[denom == 0] = 1.0
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)

def accuracy_pct(y_true, y_pred) -> float:
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = np.where(y_true > 0, y_true, np.nan)
    return float(np.nanmean((1.0 - (np.abs(y_pred - y_true) / denom)) * 100.0))

def bias_pct(y_true, y_pred) -> float:
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = np.where(y_true > 0, y_true, np.nan)
    return float(np.nanmean(((y_pred - y_true) / denom) * 100.0))

def clamp_nonneg(a):
    a = np.array(a, dtype=float)
    a[~np.isfinite(a)] = 0.0
    return np.clip(a, 0.0, None)

def _z_from_alpha(alpha: float) -> float:
    if abs(alpha - 0.10) < 1e-9: return 1.6448536269514722
    if abs(alpha - 0.05) < 1e-9: return 1.959963984540054
    if abs(alpha - 0.01) < 1e-9: return 2.5758293035489004
    return 1.959963984540054

def expm1_safe(x, cap_original: Optional[float] = None):
    a = np.array(x, dtype=float)
    a[~np.isfinite(a)] = -50.0
    a = np.maximum(a, -50.0)
    if cap_original and np.isfinite(cap_original) and cap_original > 0:
        log_cap = np.log1p(cap_original)
        a = np.minimum(a, log_cap)
    y = np.expm1(a)
    if cap_original and np.isfinite(cap_original) and cap_original > 0:
        y = np.minimum(y, cap_original)
    return np.clip(y, 0, None)

def compute_dynamic_cap(ts_m: pd.Series) -> float:
    if ts_m.empty or (ts_m.max() <= 0):
        return np.inf
    m12 = float(ts_m.tail(12).mean()) if len(ts_m) >= 3 else float(ts_m.mean())
    med, mx = float(ts_m.median()), float(ts_m.max())
    base = max(1.0, m12, med, 1.1 * mx)
    return base * 6.0

def compute_pi_from_log_mean_std(mean_log: np.ndarray, std_log: np.ndarray, cap: float, alpha: float = 0.05):
    z = _z_from_alpha(alpha)
    std_log = np.clip(np.array(std_log, dtype=float), 1e-8, None)
    lo_log = mean_log - z * std_log
    hi_log = mean_log + z * std_log
    mean = expm1_safe(mean_log, cap_original=cap)
    lo   = expm1_safe(lo_log,   cap_original=cap)
    hi   = expm1_safe(hi_log,   cap_original=cap)
    mean = clamp_nonneg(mean); lo = clamp_nonneg(lo); hi = clamp_nonneg(hi)
    return mean, np.minimum(lo, hi), np.maximum(lo, hi)

def coverage_95(actual: np.ndarray, lo: np.ndarray, hi: np.ndarray) -> float:
    a = np.array(actual, dtype=float)
    l = np.array(lo, dtype=float)
    h = np.array(hi, dtype=float)
    m = np.isfinite(a) & np.isfinite(l) & np.isfinite(h)
    if m.sum() == 0:
        return np.nan
    ok = (a[m] >= l[m]) & (a[m] <= h[m])
    return float(ok.mean() * 100.0)

def clean_outliers_daily(g: pd.DataFrame) -> pd.DataFrame:
    g = g.copy()
    s = g.sort_values("Date")["ticket_total"].astype(float)
    ql = s.quantile(0.01)
    qh = s.quantile(0.99)
    g["ticket_total"] = s.clip(ql, qh).values
    return g

def winsorize_monthly(ts_m: pd.Series, lower_q=0.01, upper_q=0.99) -> pd.Series:
    if ts_m.empty:
        return ts_m
    return ts_m.clip(lower=ts_m.quantile(lower_q), upper=ts_m.quantile(upper_q))

# ---------- Loaders ----------
def load_incoming(path: str, sheet_name: str) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet_name, engine="openpyxl")
    required = {"Date", "department_id"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Incoming missing columns: {sorted(list(missing))}. Found: {list(df.columns)}")
    if "ticket_total" not in df.columns:
        candidates = [c for c in ["total_incoming","incoming_total","Incoming"] if c in df.columns]
        if candidates:
            df["ticket_total"] = pd.to_numeric(df[candidates[0]], errors="coerce").fillna(0.0)
        elif {"incoming_from_customers","incoming_from_transfers"}.issubset(df.columns):
            df["ticket_total"] = (
                pd.to_numeric(df["incoming_from_customers"], errors="coerce").fillna(0.0)
                + pd.to_numeric(df["incoming_from_transfers"], errors="coerce").fillna(0.0)
            )
        else:
            raise ValueError("Incoming must contain ticket_total or known alternative columns.")
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])
    df["department_id"] = df["department_id"].astype(str).str.strip()
    df["ticket_total"] = pd.to_numeric(df["ticket_total"], errors="coerce").fillna(0.0).astype(float)
    if "language" not in df.columns:
        df["language"] = "English"
    df["language"] = df["language"].astype(str).str.strip()
    df["language"] = np.where(df["language"].isin(SUPPORTED_LANGS), df["language"], "English")
    if "department_name" not in df.columns:
        df["department_name"] = None
    if "vertical" not in df.columns:
        df["vertical"] = None
    return df

def load_dept_map(path: str, sheet: str) -> pd.DataFrame:
    mp = pd.read_excel(path, sheet_name=sheet, engine="openpyxl")
    rename_map = {"dept_id":"department_id","dept_name":"department_name","name":"department_name",
                  "segment":"vertical","vertical_name":"vertical"}
    mp = mp.rename(columns={k:v for k,v in rename_map.items() if k in mp.columns})
    if "department_id" not in mp.columns:
        raise ValueError(f"Department map must contain department_id. Found: {list(mp.columns)}")
    mp["department_id"] = mp["department_id"].astype(str).str.strip()
    if "department_name" in mp.columns:
        mp["department_name"] = mp["department_name"].astype(str).str.strip()
    if "vertical" in mp.columns:
        mp["vertical"] = mp["vertical"].astype(str).str.strip()
    cols = [c for c in ["department_id","department_name","vertical"] if c in mp.columns]
    return mp[cols].drop_duplicates("department_id")

def apply_mapping(incoming: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    df = incoming.merge(mapping, on="department_id", how="left", suffixes=("", "_map"))
    if "department_name_map" in df.columns:
        df["department_name"] = df["department_name"].fillna(df["department_name_map"])
    if "vertical_map" in df.columns:
        df["vertical"] = df["vertical"].fillna(df["vertical_map"])
    df["department_name"] = df["department_name"].fillna("Unknown")
    df["vertical"] = df["vertical"].fillna("Unmapped")
    df.drop(columns=[c for c in df.columns if c.endswith("_map")], inplace=True, errors="ignore")
    return df

def load_productivity(path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df = pd.read_excel(path, engine="openpyxl")
    req = {"Date","agent_id","department_id","prod_total_model"}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"productivity missing columns: {sorted(list(missing))}. Found: {list(df.columns)}")
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])
    df["department_id"] = df["department_id"].astype(str).str.strip()
    df["prod_total_model"] = pd.to_numeric(df["prod_total_model"], errors="coerce")
    df = df[np.isfinite(df["prod_total_model"]) & (df["prod_total_model"] >= 0)].copy()
    df["dow"] = df["Date"].dt.weekday.astype(int)
    dept_prod = (df.groupby("department_id", as_index=False)["prod_total_model"]
                   .mean()
                   .rename(columns={"prod_total_model":"avg_tickets_per_agent_day"}))
    dept_dow_prod = (df.groupby(["department_id","dow"], as_index=False)["prod_total_model"]
                       .mean()
                       .rename(columns={"prod_total_model":"avg_tickets_per_agent_day_dow"}))
    return dept_prod, dept_dow_prod

# ---------- Exogenous calendar ----------
def _easter_sunday(year: int) -> date:
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19*a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    L = (32 + 2*e + 2*i - h - k) % 7
    m = (a + 11*h + 22*L) // 451
    month = (h + L - 7*m + 114) // 31
    day = ((h + L - 7*m + 114) % 31) + 1
    return date(year, month, day)

def _last_friday_of_november(year: int) -> date:
    d = date(year, 11, 30)
    while d.weekday() != 4:
        d = d.replace(day=d.day - 1)
    return d

def build_eu_core_holidays(start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.DataFrame:
    years = list(range(start_date.year, end_date.year + 1))
    rows = []
    for y in years:
        rows += [
            {"ds": date(y,1,1), "type":"holiday", "weight":1.0},
            {"ds": date(y,5,1), "type":"holiday", "weight":1.0},
            {"ds": date(y,12,25), "type":"holiday", "weight":1.0},
            {"ds": date(y,12,26), "type":"holiday", "weight":1.0},
        ]
        easter = _easter_sunday(y)
        rows += [
            {"ds": easter - timedelta(days=2), "type":"holiday", "weight":1.0},
            {"ds": easter + timedelta(days=1), "type":"holiday", "weight":1.0},
        ]
        bf = _last_friday_of_november(y)
        rows += [
            {"ds": bf, "type":"event", "weight":0.6},
            {"ds": bf + timedelta(days=4), "type":"event", "weight":0.6},
        ]
    exo = pd.DataFrame(rows)
    exo["ds"] = pd.to_datetime(exo["ds"])
    return exo[(exo["ds"] >= start_date) & (exo["ds"] <= end_date)].reset_index(drop=True)

def build_exogenous_calendar(incoming: pd.DataFrame, horizon_days: int):
    start = incoming["Date"].min() - pd.Timedelta(days=365)
    end = incoming["Date"].max() + pd.Timedelta(days=horizon_days)
    exo = build_eu_core_holidays(start, end)
    cal = pd.DataFrame({"ds": pd.date_range(start, end, freq="D")})
    exo_daily = cal.merge(exo, on="ds", how="left")
    exo_daily["is_holiday"] = (exo_daily["type"] == "holiday").astype(int)
    exo_daily["is_event"] = (exo_daily["type"] == "event").astype(int)
    exo_daily["weight_h"] = np.where(exo_daily["is_holiday"] == 1, exo_daily["weight"].fillna(1.0), 0.0)
    exo_daily["weight_e"] = np.where(exo_daily["is_event"] == 1, exo_daily["weight"].fillna(1.0), 0.0)
    exo_daily = exo_daily[["ds","is_holiday","is_event","weight_h","weight_e"]]
    exo_daily["month"] = exo_daily["ds"].dt.to_period("M")
    exo_monthly = (exo_daily.groupby("month", as_index=False)
                   .agg(hol_count=("is_holiday","sum"),
                        evt_count=("is_event","sum"),
                        hol_weight_sum=("weight_h","sum"),
                        evt_weight_sum=("weight_e","sum")))
    exo_monthly["month"] = pd.PeriodIndex(exo_monthly["month"], freq="M")
    return exo_daily, exo_monthly

def prepare_monthly_exog(exo_monthly: pd.DataFrame, ts_index: pd.PeriodIndex) -> pd.DataFrame:
    ex = exo_monthly.copy()
    ex["month"] = pd.PeriodIndex(ex["month"], freq="M")
    ex = ex.set_index("month").reindex(ts_index).fillna(0.0)
    return ex[["hol_count","evt_count","hol_weight_sum","evt_weight_sum"]]

# ---------- Stability ----------
def detect_structural_break(ts_m: pd.Series, window: int = 6, min_total_months: int = 18, z_thresh: float = 2.5) -> Optional[pd.Period]:
    ts_m = ts_m.dropna()
    if len(ts_m) < max(min_total_months, 2*window + 1):
        return None
    last = ts_m.iloc[-window:]
    prev = ts_m.iloc[-2*window:-window]
    mu1, mu0 = float(last.mean()), float(prev.mean())
    s1, s0 = float(last.std(ddof=1)), float(prev.std(ddof=1))
    pooled = math.sqrt(max(1e-9, (s1*s1 + s0*s0) / 2.0))
    z = abs(mu1 - mu0) / pooled if pooled > 0 else 0.0
    return ts_m.index[-window] if z >= z_thresh else None

def apply_break_window(ts_m: pd.Series, break_start: Optional[pd.Period], min_train_months: int = 12) -> pd.Series:
    if break_start is None:
        return ts_m
    ts2 = ts_m.loc[break_start:]
    return ts2 if len(ts2) >= min_train_months else ts_m

def classify_series(ts_m: pd.Series) -> Dict[str, object]:
    ts_m = ts_m.dropna()
    n = int(len(ts_m))
    zeros_share = float((ts_m <= 0).mean()) if n > 0 else 1.0
    mean = float(ts_m.mean()) if n > 0 else 0.0
    std = float(ts_m.std(ddof=1)) if n > 1 else 0.0
    cv = float(std / mean) if mean > 0 else np.inf
    brk = detect_structural_break(ts_m, window=6, min_total_months=18, z_thresh=STAB["broken_z_thresh"])
    label = "Stable"
    reasons=[]
    if n < STAB["low_data_min_months"]:
        label="LowData"; reasons.append("low_history")
    else:
        if zeros_share >= STAB["zero_share_thresh"]:
            label="Volatile"; reasons.append("many_zeros")
        if cv >= STAB["volatility_cv_thresh"]:
            label="Volatile"; reasons.append("high_cv")
        if brk is not None:
            label="Broken"; reasons.append(f"break@{brk}")
    return {"label": label, "months": n, "zeros_share": zeros_share, "cv": cv,
            "break_start": str(brk) if brk is not None else None,
            "reasons": ";".join(reasons) if reasons else None}

# ---------- Models ----------
def fit_prophet_monthly_log_with_pi(ts_m: pd.Series, exo_m: Optional[pd.DataFrame] = None, changepoint_prior_scale: float = 0.25):
    if Prophet is None or len(ts_m) < 6:
        return None, None, None
    cap = compute_dynamic_cap(ts_m)
    y = np.log1p(ts_m.values)
    dfp = pd.DataFrame({"ds": ts_m.index.to_timestamp(), "y": y})
    if exo_m is not None:
        dfp = dfp.join(exo_m, on=ts_m.index).reset_index(drop=True)
    m = Prophet(weekly_seasonality=False, yearly_seasonality=True, daily_seasonality=False,
                interval_width=(1.0 - PI_ALPHA), changepoint_prior_scale=changepoint_prior_scale)
    if exo_m is not None:
        for col in exo_m.columns:
            m.add_regressor(col)
    m.fit(dfp)

    def _future(h_months: int):
        future = m.make_future_dataframe(periods=h_months, freq="MS")
        if exo_m is not None:
            idx_future = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
            pad = pd.concat([exo_m.iloc[[-1]]] * h_months, ignore_index=True)
            pad.index = idx_future
            ex_full = pd.concat([exo_m, pad], axis=0)
            for col in exo_m.columns:
                future[col] = ex_full[col].values
        return future

    def f_pi(h_months: int):
        pred = m.predict(_future(h_months))
        pred.index = pd.PeriodIndex(pred["ds"], freq="M")
        idx = pred.index[-h_months:]
        mu_log = pred["yhat"].iloc[-h_months:].values
        lo_log = pred["yhat_lower"].iloc[-h_months:].values
        hi_log = pred["yhat_upper"].iloc[-h_months:].values
        mean = clamp_nonneg(expm1_safe(mu_log, cap_original=cap))
        lo = clamp_nonneg(expm1_safe(lo_log, cap_original=cap))
        hi = clamp_nonneg(expm1_safe(hi_log, cap_original=cap))
        return pd.Series(mean, index=idx), pd.Series(np.minimum(lo,hi), index=idx), pd.Series(np.maximum(lo,hi), index=idx)

    def f_mean(h_months: int):
        mean, _, _ = f_pi(h_months)
        return mean

    return m, f_mean, f_pi

def fit_arima_monthly_log_with_pi(ts_m: pd.Series, exo_m: Optional[pd.DataFrame] = None):
    cap = compute_dynamic_cap(ts_m)
    y = np.log1p(ts_m)
    best_aic, best_model, best_exog, best_order = np.inf, None, None, None
    pqs=[0,1]
    seasonal = len(ts_m) >= 12
    PsQs=[0]
    for p in pqs:
        for d in ([1] if len(ts_m) < 24 else [0,1]):
            for q in pqs:
                for P in PsQs:
                    for D in ([0,1] if seasonal else [0]):
                        for Q in PsQs:
                            try:
                                model = SARIMAX(
                                    y,
                                    order=(p,d,q),
                                    seasonal_order=(P,D,Q,12 if seasonal else 0),
                                    exog=(exo_m.values if exo_m is not None else None),
                                    enforce_stationarity=False,
                                    enforce_invertibility=False
                                ).fit(disp=False)
                                if hasattr(model, "mle_retvals") and not bool(model.mle_retvals.get("converged", True)):
                                    continue
                                if model.aic < best_aic:
                                    best_aic, best_model, best_exog = model.aic, model, exo_m
                                    best_order = ((p,d,q),(P,D,Q,12 if seasonal else 0))
                            except Exception:
                                continue

    def _future_exog(h_months: int, idx: pd.PeriodIndex):
        if best_exog is None: return None
        pad = pd.concat([best_exog.iloc[[-1]]] * h_months, ignore_index=True)
        pad.index = idx
        return pad.values

    def f_pi(h_months: int):
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
        if best_model is None:
            mu_log = np.full(h_months, float(np.nanmean(y.values)))
            std_log = np.full(h_months, float(np.nanstd(y.values) if np.isfinite(np.nanstd(y.values)) else 0.5))
            mean, lo, hi = compute_pi_from_log_mean_std(mu_log, std_log, cap=cap, alpha=PI_ALPHA)
            return pd.Series(mean,index=idx), pd.Series(lo,index=idx), pd.Series(hi,index=idx)
        fc = best_model.get_forecast(h_months, exog=_future_exog(h_months, idx))
        mu_log = fc.predicted_mean.values
        ci = fc.conf_int(alpha=PI_ALPHA)
        lo_log = ci.iloc[:,0].values
        hi_log = ci.iloc[:,1].values
        mean = clamp_nonneg(expm1_safe(mu_log, cap_original=cap))
        lo = clamp_nonneg(expm1_safe(lo_log, cap_original=cap))
        hi = clamp_nonneg(expm1_safe(hi_log, cap_original=cap))
        return pd.Series(mean,index=idx), pd.Series(np.minimum(lo,hi),index=idx), pd.Series(np.maximum(lo,hi),index=idx)

    return best_model, f_pi, best_order

def fit_ets_monthly_log_with_pi(ts_m: pd.Series):
    cap = compute_dynamic_cap(ts_m)
    y_log = np.log1p(ts_m)
    seasonal = 12 if len(ts_m) >= 24 else None
    model = ExponentialSmoothing(y_log, trend="add", seasonal=("add" if seasonal else None), seasonal_periods=seasonal).fit()
    resid = np.array(y_log.values, dtype=float) - np.array(model.fittedvalues, dtype=float)
    sigma = float(np.nanstd(resid)) if np.isfinite(np.nanstd(resid)) else 0.5

    def f_pi(h_months: int):
        vals_log = np.array(model.forecast(h_months), dtype=float)
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
        mean, lo, hi = compute_pi_from_log_mean_std(vals_log, np.full(h_months, sigma), cap=cap, alpha=PI_ALPHA)
        return pd.Series(mean,index=idx), pd.Series(lo,index=idx), pd.Series(hi,index=idx)

    return model, f_pi

def rolling_cv_monthly(ts_m: pd.Series, exo_m: Optional[pd.DataFrame]):
    n = len(ts_m)
    if n < 9: return None
    h = 3 if n >= 15 else 1
    min_train = max(12, n - (h + 2))
    splits=[]
    for start in range(min_train, n - h + 1):
        train = ts_m.iloc[:start]
        test  = ts_m.iloc[start:start+h]
        ex_train = exo_m.iloc[:start] if exo_m is not None else None
        m={}
        mp, _, fp_pi = fit_prophet_monthly_log_with_pi(train, ex_train, changepoint_prior_scale=0.30)
        if fp_pi is not None:
            try:
                yhat = fp_pi(h)[0].values[:h]
                m["Prophet"] = smape(test.values, yhat)
            except Exception:
                m["Prophet"]=200.0
        try:
            _, fa_pi, _ = fit_arima_monthly_log_with_pi(train, ex_train)
            yhat = fa_pi(h)[0].values[:h]
            m["ARIMA"] = smape(test.values, yhat)
        except Exception:
            m["ARIMA"]=200.0
        try:
            _, fe_pi = fit_ets_monthly_log_with_pi(train)
            yhat = fe_pi(h)[0].values[:h]
            m["ETS"] = smape(test.values, yhat)
        except Exception:
            m["ETS"]=200.0
        splits.append(m)
    return pd.DataFrame(splits).mean().to_dict()

def select_or_blend_with_pi(fc_pi: Dict[str, Tuple[pd.Series,pd.Series,pd.Series]], cv_scores: Dict[str,float], blend: bool):
    keys = list(fc_pi.keys())
    if not keys:
        raise ValueError("No candidates.")
    scores = {k: float(v) for k,v in (cv_scores or {}).items() if k in keys and np.isfinite(v)}
    if (not blend) or (len(scores)==0):
        k = keys[0]
        mean, lo, hi = fc_pi[k]
        return mean, lo, hi, {"winner": k, "weights": {k:1.0}}

    inv = {k: (1.0/v if v > 0 else 0.0) for k,v in scores.items()}
    total = sum(inv.values())
    if total <= 0:
        k = min(scores, key=scores.get)
        mean, lo, hi = fc_pi[k]
        return mean, lo, hi, {"winner": k, "weights": {k:1.0}}
    w = {k: inv[k]/total for k in inv}
    idx = None
    for k in keys:
        idx = fc_pi[k][0].index if idx is None else idx.union(fc_pi[k][0].index)

    mean_bl = sum(w.get(k,0.0) * fc_pi[k][0].reindex(idx).fillna(0.0) for k in keys)

    z = _z_from_alpha(PI_ALPHA)
    var_log = np.zeros(len(idx), dtype=float)
    for k, wk in w.items():
        mu, lo, hi = fc_pi[k]
        lo_v = lo.reindex(idx).astype(float).values
        hi_v = hi.reindex(idx).astype(float).values
        lo_log = np.log1p(np.clip(lo_v,0,None))
        hi_log = np.log1p(np.clip(hi_v,0,None))
        sigma = (hi_log - lo_log) / (2.0*z)
        sigma = np.clip(sigma, 1e-8, None)
        var_log += (wk**2) * (sigma**2)
    std_log = np.sqrt(np.clip(var_log, 1e-10, None))
    mu_v = mean_bl.reindex(idx).astype(float).values
    mu_log = np.log1p(np.clip(mu_v,0,None))
    cap = float(np.nanmax(mu_v) * 6.0) if np.isfinite(np.nanmax(mu_v)) and np.nanmax(mu_v)>0 else np.inf
    mean, lo, hi = compute_pi_from_log_mean_std(mu_log, std_log, cap=cap, alpha=PI_ALPHA)
    return pd.Series(mean,index=idx), pd.Series(lo,index=idx), pd.Series(hi,index=idx), {"winner": min(scores, key=scores.get), "weights": w}

# ---------- Hierarchical shares ----------
def dept_share_roll3_within_vertical(incoming: pd.DataFrame) -> pd.DataFrame:
    d = incoming.copy()
    d["Date"] = pd.to_datetime(d["Date"])
    d["month"] = d["Date"].dt.to_period("M")
    dept_m = d.groupby(["vertical","department_id","month"], as_index=False)["ticket_total"].sum()
    vert_m = d.groupby(["vertical","month"], as_index=False)["ticket_total"].sum().rename(columns={"ticket_total":"vertical_total"})
    x = dept_m.merge(vert_m, on=["vertical","month"], how="left")
    x["share"] = np.where(x["vertical_total"]>0, x["ticket_total"]/x["vertical_total"], 0.0)
    x = x.sort_values(["vertical","department_id","month"])
    x["share_roll3"] = (x.groupby(["vertical","department_id"])["share"]
                          .rolling(3, min_periods=1).mean()
                          .reset_index(level=[0,1], drop=True))
    return x[["vertical","department_id","month","share_roll3"]]

def normalize_shares_for_month(shares_vd: pd.DataFrame, vertical: str, month: pd.Period, dept_ids: np.ndarray) -> pd.DataFrame:
    sub = shares_vd[(shares_vd["vertical"]==vertical) & (shares_vd["month"]==month)].copy()
    grid = pd.DataFrame({"department_id": [str(x) for x in dept_ids]})
    sub = grid.merge(sub[["department_id","share_roll3"]], on="department_id", how="left").fillna(0.0)
    s = float(sub["share_roll3"].sum())
    if s <= 0:
        sub["share_norm"] = 1.0 / max(1, len(sub))
    else:
        sub["share_norm"] = sub["share_roll3"] / s
    return sub[["department_id","share_norm"]]

# ---------- Level adjustment on dept forecasts ----------
def apply_level_adjustment(fc_dept: pd.DataFrame, incoming: pd.DataFrame) -> pd.DataFrame:
    out = fc_dept.copy()
    if not LEVEL_ADJ["enabled"] or out.empty:
        out["level_adj_factor"] = 1.0
        return out
    d = incoming.copy()
    d["month"] = pd.to_datetime(d["Date"]).dt.to_period("M")
    hist = d.groupby(["department_id","month"], as_index=False)["ticket_total"].sum()
    hist["month"] = pd.PeriodIndex(hist["month"], freq="M")

    k = int(LEVEL_ADJ["lookback_months"])
    clip_min = float(LEVEL_ADJ["clip_min"]); clip_max = float(LEVEL_ADJ["clip_max"])
    factors={}
    for dept, g in hist.groupby("department_id"):
        g = g.sort_values("month")
        if len(g) < k*2:
            factors[str(dept)] = 1.0
            continue
        recent = float(g["ticket_total"].tail(k).mean())
        baseline = float(g["ticket_total"].tail(k*2).head(k).mean())  # previous k months
        if baseline <= 0:
            factors[str(dept)] = 1.0
        else:
            factors[str(dept)] = float(np.clip(recent / baseline, clip_min, clip_max))

    out["level_adj_factor"] = out["department_id"].astype(str).map(factors).fillna(1.0)
    for col in ["forecast_monthly_dept","forecast_p05_dept","forecast_p95_dept"]:
        out[col] = out[col].astype(float) * out["level_adj_factor"].astype(float)
    return out

# ---------- Language shares (unchanged) ----------
def compute_language_shares_rolling3(incoming: pd.DataFrame) -> pd.DataFrame:
    d = incoming.copy()
    d["Date"] = pd.to_datetime(d["Date"])
    d["month"] = d["Date"].dt.to_period("M")
    d["language"] = d["language"].astype(str).str.strip()
    d["language"] = np.where(d["language"].isin(SUPPORTED_LANGS), d["language"], "English")
    agg = (d.groupby(["department_id","month","language"], as_index=False)["ticket_total"].sum()
             .sort_values(["department_id","language","month"]))
    totals = agg.groupby(["department_id","month"], as_index=False)["ticket_total"].sum().rename(columns={"ticket_total":"dept_total"})
    agg = agg.merge(totals, on=["department_id","month"], how="left")
    agg["share"] = np.where(agg["dept_total"] > 0, agg["ticket_total"]/agg["dept_total"], 0.0)
    agg["share_roll3"] = (agg.groupby(["department_id","language"])["share"]
                            .rolling(3, min_periods=1).mean()
                            .reset_index(level=[0,1], drop=True))
    agg = agg[agg["language"].isin(SUPPORTED_LANGS)].copy()
    return agg[["department_id","month","language","share_roll3"]]

def apply_language_split(fc_dept: pd.DataFrame, shares: pd.DataFrame) -> pd.DataFrame:
    f = fc_dept.copy()
    f["month"] = pd.PeriodIndex(f["month"], freq="M")
    lang_df = pd.DataFrame({"language": SUPPORTED_LANGS})
    f_exp = f.merge(lang_df, how="cross")
    sh = shares.copy()
    sh["month"] = pd.PeriodIndex(sh["month"], freq="M")
    f_exp = f_exp.merge(sh, on=["department_id","month","language"], how="left")
    f_exp["share_roll3"] = f_exp["share_roll3"].fillna(0.0)
    sums = f_exp.groupby(["department_id","month"])["share_roll3"].transform("sum")
    f_exp["share_norm"] = np.where(sums > 0, f_exp["share_roll3"]/sums, 0.0)
    f_exp["forecast_monthly"] = f_exp["forecast_monthly_dept"].astype(float) * f_exp["share_norm"].astype(float)
    f_exp["forecast_p05"] = f_exp["forecast_p05_dept"].astype(float) * f_exp["share_norm"].astype(float)
    f_exp["forecast_p95"] = f_exp["forecast_p95_dept"].astype(float) * f_exp["share_norm"].astype(float)
    keep = ["department_id","language","month","forecast_monthly","forecast_p05","forecast_p95","share_norm",
            "stability_label","winner_model","models_used","level_adj_factor"]
    return f_exp[keep]

# ---------- Staffing + daily plan utilities ----------
def staffing_rule(accuracy: float, f: float, p95: float) -> float:
    if not np.isfinite(accuracy):
        return p95
    if accuracy < RISK_POLICY["critical_low_acc_use_p95"]:
        return p95
    if accuracy < RISK_POLICY["critical_mid_acc_blend"]:
        return f + 0.5*(p95 - f)
    return f

def uplift_from_label(label: str) -> float:
    if label == "Broken": return RISK_POLICY["broken_uplift_pct"]
    if label == "LowData": return RISK_POLICY["lowdata_uplift_pct"]
    if label == "Volatile": return RISK_POLICY["volatility_uplift_pct"]
    return 0.0

def get_weekend_service_policy(mapping: pd.DataFrame) -> pd.DataFrame:
    mp = mapping.copy()
    mp["vertical"] = mp.get("vertical", "Unmapped").astype(str).str.strip()
    mp["department_name"] = mp.get("department_name", "Unknown").astype(str).str.strip()
    payments_no_weekend_names = {"CA_PYAC","L2 Customer Support","Datatrans L2 Customer Support","Specialist - L2 Customer Support"}
    mp["weekend_service"] = True
    mp.loc[mp["vertical"].str.lower() == "partners", "weekend_service"] = False
    is_payments = mp["vertical"].str.lower() == "payments"
    mp.loc[is_payments & mp["department_name"].isin(payments_no_weekend_names), "weekend_service"] = False
    return mp[["department_id","vertical","department_name","weekend_service"]].drop_duplicates("department_id")

def compute_dept_dow_profile(incoming: pd.DataFrame, lookback_days: int = 180, min_obs: int = 30) -> pd.DataFrame:
    d = incoming.copy()
    d["Date"] = pd.to_datetime(d["Date"])
    maxd = d["Date"].max()
    d = d[d["Date"] >= (maxd - pd.Timedelta(days=lookback_days))].copy()
    d["dow"] = d["Date"].dt.weekday.astype(int)
    agg = d.groupby(["department_id","dow"], as_index=False)["ticket_total"].sum()
    totals = agg.groupby("department_id", as_index=False)["ticket_total"].sum().rename(columns={"ticket_total":"dept_total"})
    agg = agg.merge(totals, on="department_id", how="left")
    agg["weight_raw"] = np.where(agg["dept_total"] > 0, agg["ticket_total"]/agg["dept_total"], 0.0)
    depts = d["department_id"].astype(str).unique().tolist()
    grid = pd.MultiIndex.from_product([depts, list(range(7))], names=["department_id","dow"]).to_frame(index=False)
    prof = grid.merge(agg[["department_id","dow","weight_raw"]], on=["department_id","dow"], how="left").fillna(0.0)
    obs = d.groupby("department_id")["Date"].count().rename("n_obs").reset_index()
    prof = prof.merge(obs, on="department_id", how="left").fillna({"n_obs":0})
    def _fallback():
        w = np.zeros(7, dtype=float); w[:5] = 1.0/5.0
        return w
    out=[]
    for dept, g in prof.groupby("department_id"):
        n = int(g["n_obs"].iloc[0])
        if n < min_obs or float(g["weight_raw"].sum()) <= 0:
            w = _fallback()
        else:
            w = g.sort_values("dow")["weight_raw"].values.astype(float)
            s = float(w.sum()); w = w/s if s>0 else _fallback()
        for dow, wv in enumerate(w):
            out.append({"department_id": str(dept), "dow": int(dow), "dow_weight": float(wv)})
    return pd.DataFrame(out)

def build_daily_plan_from_monthly_staffing(staff_m: pd.DataFrame, incoming_hist: pd.DataFrame,
                                           mapping_policy: pd.DataFrame, dept_dow_profile: pd.DataFrame,
                                           horizon_days: int) -> pd.DataFrame:
    last_date = incoming_hist["Date"].max()
    start = last_date + pd.Timedelta(days=1)
    end = start + pd.Timedelta(days=horizon_days - 1)
    wknd = dict(zip(mapping_policy["department_id"].astype(str), mapping_policy["weekend_service"].astype(bool)))
    prof_key = {(r["department_id"], int(r["dow"])): float(r["dow_weight"]) for _, r in dept_dow_profile.iterrows()}
    rows=[]
    for _, r in staff_m.iterrows():
        dept = str(r["department_id"])
        m = r["month"]; m = m if isinstance(m, pd.Period) else pd.Period(m, freq="M")
        month_days = pd.date_range(m.start_time, m.end_time, freq="D")
        month_days = month_days[(month_days >= start) & (month_days <= end)]
        if len(month_days)==0: 
            continue
        weekend_service = bool(wknd.get(dept, True))
        w=[]
        for dday in month_days:
            dow = int(dday.weekday())
            if (not weekend_service) and (dow>=5): w.append(0.0)
            else: w.append(float(prof_key.get((dept, dow), 0.0)))
        w=np.array(w, dtype=float); s=float(w.sum())
        if s<=0:
            allowed = np.array([(dday.weekday() < 5) if (not weekend_service) else True for dday in month_days], dtype=bool)
            w=allowed.astype(float); s=float(w.sum()) if float(w.sum())>0 else 1.0
        w=w/s
        monthly_total=float(r["staffing_volume_monthly"])
        daily_alloc=monthly_total*w
        for dday, val in zip(month_days, daily_alloc):
            rows.append({"Date": dday, "department_id": dept,
                         "vertical": r.get("vertical"), "department_name": r.get("department_name"),
                         "weekend_service": weekend_service,
                         "Staffing_Daily_Tickets": float(max(0.0, val))})
    return pd.DataFrame(rows)


## 4) Hierarchical monthly forecasting (Vertical → Dept) + Accuracy + Staffing

In [4]:
# ---------- Vertical forecast ----------
def forecast_monthly_by_vertical(incoming: pd.DataFrame, exo_monthly: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    rows=[]
    st_rows=[]
    for vert, g in incoming.groupby("vertical"):
        g = clean_outliers_daily(g.sort_values("Date"))
        vm = g.assign(month=g["Date"].dt.to_period("M")).groupby("month")["ticket_total"].sum().sort_index()
        vm.index = pd.PeriodIndex(vm.index, freq="M")
        if len(vm)==0:
            continue
        vm_full = winsorize_monthly(vm)
        st = classify_series(vm_full)
        st_rows.append({"vertical": vert, **st})

        brk = detect_structural_break(vm_full, z_thresh=STAB["broken_z_thresh"])
        train = apply_break_window(vm_full, brk, min_train_months=ACCURACY_MIN_TRAIN_MONTHS)
        ex_train = prepare_monthly_exog(exo_monthly, train.index)

        label = st["label"]
        blend = (label == "Stable")
        prophet_cp = 0.25 if label == "Stable" else (0.35 if label == "Volatile" else 0.45)
        allow_arima = (label == "Stable") and (vert not in CRITICAL_VERTICALS)

        fc_pi={}
        used=[]
        winner=None

        # Prophet
        mp, _, fp_pi = fit_prophet_monthly_log_with_pi(train, ex_train, changepoint_prior_scale=prophet_cp)
        if fp_pi is not None:
            fc_pi["Prophet"] = fp_pi(H_MONTHS); used.append("Prophet")
        # ETS
        try:
            _, fe_pi = fit_ets_monthly_log_with_pi(train)
            fc_pi["ETS"] = fe_pi(H_MONTHS); used.append("ETS")
        except Exception:
            pass
        # ARIMA (optional)
        if allow_arima:
            try:
                _, fa_pi, _ = fit_arima_monthly_log_with_pi(train, ex_train)
                if fa_pi is not None:
                    fc_pi["ARIMA"] = fa_pi(H_MONTHS); used.append("ARIMA")
            except Exception:
                pass

        if not fc_pi:
            idx = pd.period_range(train.index[-1] + 1, periods=H_MONTHS, freq="M")
            val = max(0.0, float(train.mean()))
            fc_pi["NaiveMean"] = (pd.Series([val]*H_MONTHS,index=idx),
                                 pd.Series([val]*H_MONTHS,index=idx),
                                 pd.Series([val]*H_MONTHS,index=idx))
            used.append("NaiveMean")

        cv = rolling_cv_monthly(train, ex_train) or {}
        mean, p05, p95, meta = select_or_blend_with_pi(fc_pi, cv_scores=cv, blend=blend)
        winner = meta.get("winner")

        for per in mean.index:
            rows.append({
                "vertical": vert,
                "month": per,
                "forecast_monthly_vertical": float(mean.loc[per]),
                "forecast_p05_vertical": float(p05.loc[per]),
                "forecast_p95_vertical": float(p95.loc[per]),
                "stability_label": label,
                "models_used": ",".join(used),
                "winner_model": winner,
                "prophet_cp": float(prophet_cp),
                "break_start": st.get("break_start"),
            })

    fc_v = pd.DataFrame(rows)
    if not fc_v.empty:
        fc_v["month"] = pd.PeriodIndex(fc_v["month"], freq="M")
    st_v = pd.DataFrame(st_rows)
    return fc_v, st_v


# ---------- Allocation vertical -> dept ----------
def allocate_vertical_to_dept(fc_vertical: pd.DataFrame, incoming: pd.DataFrame, shares_vd: pd.DataFrame) -> pd.DataFrame:
    rows=[]
    inc = incoming.copy()
    inc["month"] = pd.to_datetime(inc["Date"]).dt.to_period("M")
    depts_by_vert = inc.groupby("vertical")["department_id"].apply(lambda s: sorted(set(s.astype(str)))).to_dict()

    for _, r in fc_vertical.iterrows():
        vert = r["vertical"]; per = r["month"]
        dept_ids = np.array(depts_by_vert.get(vert, []), dtype=object)
        if len(dept_ids)==0:
            continue

        # prefer shares for same forecast month; fallback to latest available historical shares
        if (shares_vd["vertical"].eq(vert) & shares_vd["month"].eq(per)).any():
            use_month = per
        else:
            hist_sub = shares_vd[shares_vd["vertical"]==vert]
            use_month = hist_sub["month"].max() if not hist_sub.empty else per

        sh_norm = normalize_shares_for_month(shares_vd, vert, use_month, dept_ids)

        for _, sd in sh_norm.iterrows():
            dept = str(sd["department_id"])
            share = float(sd["share_norm"])
            rows.append({
                "vertical": vert,
                "department_id": dept,
                "month": per,
                "forecast_monthly_dept": float(r["forecast_monthly_vertical"] * share),
                "forecast_p05_dept": float(r["forecast_p05_vertical"] * share),
                "forecast_p95_dept": float(r["forecast_p95_vertical"] * share),
                "share_vert_to_dept": share,
                "stability_label": r.get("stability_label"),
                "models_used": r.get("models_used"),
                "winner_model": r.get("winner_model"),
                "prophet_cp": r.get("prophet_cp"),
            })

    df = pd.DataFrame(rows)
    if not df.empty:
        df["month"] = pd.PeriodIndex(df["month"], freq="M")
    return df


# ---------- Accuracy backtest (hierarchical) ----------
def backtest_accuracy_hierarchical(incoming: pd.DataFrame, exo_monthly: pd.DataFrame) -> pd.DataFrame:
    inc = incoming.copy()
    inc["Date"] = pd.to_datetime(inc["Date"])
    inc["month"] = inc["Date"].dt.to_period("M")

    dept_m = inc.groupby(["department_id","vertical","month"], as_index=False)["ticket_total"].sum()
    dept_m["month"] = pd.PeriodIndex(dept_m["month"], freq="M")
    vert_m = inc.groupby(["vertical","month"], as_index=False)["ticket_total"].sum()
    vert_m["month"] = pd.PeriodIndex(vert_m["month"], freq="M")

    shares_vd = dept_share_roll3_within_vertical(inc)

    out=[]
    for vert, vdf in vert_m.groupby("vertical"):
        vdf = vdf.sort_values("month")
        if len(vdf) < (ACCURACY_MIN_TRAIN_MONTHS + ACCURACY_BACKTEST_MONTHS):
            continue

        eval_months = vdf["month"].iloc[-ACCURACY_BACKTEST_MONTHS:].tolist()
        dept_ids = dept_m.loc[dept_m["vertical"]==vert, "department_id"].astype(str).unique().tolist()
        if not dept_ids:
            continue

        pred_map = {d: [] for d in dept_ids}
        lo_map   = {d: [] for d in dept_ids}
        hi_map   = {d: [] for d in dept_ids}
        act_map  = {d: [] for d in dept_ids}

        splits_done=0
        for m in eval_months[::-1]:
            train_end = m - 1
            train_v = vdf[vdf["month"] <= train_end].set_index("month")["ticket_total"]
            train_v.index = pd.PeriodIndex(train_v.index, freq="M")
            if len(train_v) < ACCURACY_MIN_TRAIN_MONTHS:
                continue

            ex_train = prepare_monthly_exog(exo_monthly, train_v.index)
            st = classify_series(train_v)
            label = st["label"]
            blend = (label == "Stable")
            prophet_cp = 0.30 if label == "Stable" else (0.40 if label == "Volatile" else 0.50)
            allow_arima = (label == "Stable") and (vert not in CRITICAL_VERTICALS)

            fc_pi={}
            mp, _, fp_pi = fit_prophet_monthly_log_with_pi(train_v, ex_train, changepoint_prior_scale=prophet_cp)
            if fp_pi is not None:
                fc_pi["Prophet"] = fp_pi(ACCURACY_HORIZON_MONTHS)
            try:
                _, fe_pi = fit_ets_monthly_log_with_pi(train_v)
                fc_pi["ETS"] = fe_pi(ACCURACY_HORIZON_MONTHS)
            except Exception:
                pass
            if allow_arima:
                try:
                    _, fa_pi, _ = fit_arima_monthly_log_with_pi(train_v, ex_train)
                    if fa_pi is not None:
                        fc_pi["ARIMA"] = fa_pi(ACCURACY_HORIZON_MONTHS)
                except Exception:
                    pass
            if not fc_pi:
                continue

            cv = rolling_cv_monthly(train_v, ex_train) or {}
            mean_v, p05_v, p95_v, _ = select_or_blend_with_pi(fc_pi, cv_scores=cv, blend=blend)
            if m not in mean_v.index:
                continue

            v_fore = float(mean_v.loc[m]); v_lo=float(p05_v.loc[m]); v_hi=float(p95_v.loc[m])

            sh_hist = shares_vd[(shares_vd["vertical"]==vert) & (shares_vd["month"]<=train_end)]
            if sh_hist.empty:
                sh_norm = pd.DataFrame({"department_id": dept_ids, "share_norm": [1/len(dept_ids)]*len(dept_ids)})
            else:
                use_month = sh_hist["month"].max()
                sh_norm = normalize_shares_for_month(shares_vd, vert, use_month, np.array(dept_ids, dtype=object))

            act_sub = dept_m[(dept_m["vertical"]==vert) & (dept_m["month"]==m)][["department_id","ticket_total"]].copy()
            act_sub["department_id"] = act_sub["department_id"].astype(str)
            act_dict = dict(zip(act_sub["department_id"], act_sub["ticket_total"].astype(float)))

            for _, sd in sh_norm.iterrows():
                dpt = str(sd["department_id"])
                share = float(sd["share_norm"])
                pred_map[dpt].append(v_fore * share)
                lo_map[dpt].append(v_lo * share)
                hi_map[dpt].append(v_hi * share)
                act_map[dpt].append(float(act_dict.get(dpt, 0.0)))

            splits_done += 1
            if splits_done >= ACCURACY_MAX_SPLITS:
                break

        for dpt in dept_ids:
            y_true = np.array(act_map[dpt], dtype=float)
            y_pred = np.array(pred_map[dpt], dtype=float)
            if len(y_true)==0:
                continue
            out.append({
                "department_id": str(dpt),
                "vertical": vert,
                "Eval_Months": int(len(y_true)),
                "sMAPE_%": float(smape(y_true, y_pred)),
                "MAE": float(np.mean(np.abs(y_pred - y_true))),
                "Bias_%": float(bias_pct(y_true, y_pred)),
                "Accuracy_%": float(accuracy_pct(y_true, y_pred)),
                "PI_Coverage_95_%": float(coverage_95(y_true, np.array(lo_map[dpt],dtype=float), np.array(hi_map[dpt],dtype=float)))
            })

    return pd.DataFrame(out)


# ---------- Staffing monthly ----------
def compute_staffing_monthly(fc_dept: pd.DataFrame, acc: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    df = fc_dept.merge(mapping, on="department_id", how="left", suffixes=("", "_map"))
    # Guarantee vertical/department_name columns exist after merge
    if "vertical" not in df.columns:
        df["vertical"] = None
    if "vertical_map" in df.columns:
        df["vertical"] = df["vertical"].fillna(df["vertical_map"])
        df.drop(columns=["vertical_map"], inplace=True, errors="ignore")
    if "department_name" not in df.columns:
        df["department_name"] = None
    if "department_name_map" in df.columns:
        df["department_name"] = df["department_name"].fillna(df["department_name_map"])
        df.drop(columns=["department_name_map"], inplace=True, errors="ignore")
    df["vertical"] = df["vertical"].fillna("Unmapped")
    df["department_name"] = df["department_name"].fillna("Unknown")
    if acc is None or acc.empty:
        df["Accuracy_%"] = np.nan
    else:
        df = df.merge(acc[["department_id","Accuracy_%"]], on="department_id", how="left")

    staff=[]
    uplift=[]
    for _, r in df.iterrows():
        f = float(r["forecast_monthly_dept"])
        p95 = float(r["forecast_p95_dept"]) if np.isfinite(r["forecast_p95_dept"]) else f
        accv = float(r["Accuracy_%"]) if np.isfinite(r["Accuracy_%"]) else np.nan
        label = r.get("stability_label","Stable")
        vert = r.get("vertical", None)

        base = staffing_rule(accv, f, p95) if vert in CRITICAL_VERTICALS else f
        up = uplift_from_label(label)
        staff.append(base * (1.0 + up/100.0))
        uplift.append(up)

    df["risk_uplift_pct"] = uplift
    df["staffing_volume_monthly"] = staff
    return df


## 5) Run + Export

In [5]:
incoming = load_incoming(INCOMING_SOURCE_PATH, INCOMING_SHEET)
mapping = load_dept_map(DEPT_MAP_PATH, DEPT_MAP_SHEET)
incoming = apply_mapping(incoming, mapping)

dept_prod, dept_dow_prod = load_productivity(PRODUCTIVITY_PATH)

exo_daily, exo_monthly = build_exogenous_calendar(incoming, horizon_days=DAILY_HORIZON_DAYS)

shares_vd = dept_share_roll3_within_vertical(incoming)
fc_vertical, vertical_stability = forecast_monthly_by_vertical(incoming, exo_monthly)
fc_dept = allocate_vertical_to_dept(fc_vertical, incoming, shares_vd)

# Level adjustment
fc_dept_adj = apply_level_adjustment(fc_dept, incoming)

# Accuracy (hierarchical)
acc_dept = backtest_accuracy_hierarchical(incoming, exo_monthly) if ENABLE_ACCURACY_TABLE else pd.DataFrame()
if acc_dept is not None and not acc_dept.empty:
    acc_dept = acc_dept.merge(mapping, on="department_id", how="left", suffixes=("", "_map"))
    # Hard guarantee: vertical column exists (fallback to mapped or "Unmapped")
    if "vertical" not in acc_dept.columns:
        acc_dept["vertical"] = None
    if "vertical_map" in acc_dept.columns:
        acc_dept["vertical"] = acc_dept["vertical"].fillna(acc_dept["vertical_map"])
        acc_dept.drop(columns=["vertical_map"], inplace=True, errors="ignore")
    acc_dept["vertical"] = acc_dept["vertical"].fillna("Unmapped")
    # Sort only by available columns to avoid KeyError
    sort_cols = [c for c in ["vertical","department_id"] if c in acc_dept.columns]
    acc_dept = acc_dept.sort_values(sort_cols)
if "vertical_map" in acc_dept.columns:
    acc_dept["vertical"] = acc_dept.get("vertical").fillna(acc_dept["vertical_map"])
    acc_dept.drop(columns=["vertical_map"], inplace=True, errors="ignore")
if "department_name_map" in acc_dept.columns:
    acc_dept["department_name"] = acc_dept.get("department_name").fillna(acc_dept["department_name_map"])
    acc_dept.drop(columns=["department_name_map"], inplace=True, errors="ignore")
sort_cols = [c for c in ["vertical","department_id"] if c in acc_dept.columns]
acc_dept = acc_dept.sort_values(sort_cols)

# Language split
lang_shares = compute_language_shares_rolling3(incoming)
fc_dept_lang = apply_language_split(fc_dept_adj, lang_shares).merge(mapping, on="department_id", how="left")

# Staffing
staff_m = compute_staffing_monthly(fc_dept_adj, acc_dept, mapping)
sort_cols = [c for c in ["vertical","department_id","month"] if c in staff_m.columns]
staff_m = staff_m.sort_values(sort_cols)

# Daily plan
dept_dow_profile = compute_dept_dow_profile(incoming, lookback_days=DOW_LOOKBACK_DAYS, min_obs=DOW_MIN_OBS)
dept_policy = get_weekend_service_policy(mapping)
daily_staff = build_daily_plan_from_monthly_staffing(staff_m, incoming, dept_policy, dept_dow_profile, horizon_days=DAILY_HORIZON_DAYS)

# FTE conversion via productivity
daily_staff["dow"] = pd.to_datetime(daily_staff["Date"]).dt.weekday.astype(int)
daily_staff = daily_staff.merge(dept_dow_prod, on=["department_id","dow"], how="left")
daily_staff = daily_staff.merge(dept_prod, on="department_id", how="left")
daily_staff["prod_used"] = daily_staff["avg_tickets_per_agent_day_dow"].fillna(daily_staff["avg_tickets_per_agent_day"])
daily_staff["Capacity_FTE_per_day"] = np.where(
    pd.to_numeric(daily_staff["prod_used"], errors="coerce").fillna(0.0) > 0,
    daily_staff["Staffing_Daily_Tickets"] / pd.to_numeric(daily_staff["prod_used"], errors="coerce"),
    np.nan
)

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as w:
    vertical_stability.sort_values(["vertical"]).to_excel(w, "vertical_stability", index=False)
    fc_vertical.sort_values(["vertical","month"]).to_excel(w, "forecast_vertical_monthly", index=False)
    shares_vd.sort_values(["vertical","department_id","month"]).to_excel(w, "dept_share_roll3_within_vertical", index=False)

    fc_dept_adj.merge(mapping, on="department_id", how="left").sort_values(["vertical","department_id","month"]).to_excel(w, "forecast_dept_monthly", index=False)
    fc_dept_lang.sort_values(["vertical","department_id","language","month"]).to_excel(w, "forecast_dept_lang_monthly", index=False)

    if acc_dept is not None and not acc_dept.empty:
        acc_dept.to_excel(w, "accuracy_dept_monthly", index=False)

    staff_m.to_excel(w, "staffing_monthly_dept", index=False)
    dept_dow_profile.sort_values(["department_id","dow"]).to_excel(w, "dow_profile_dept", index=False)
    daily_staff.sort_values(["vertical","department_id","Date"]).to_excel(w, "daily_capacity_plan_90d", index=False)

print("✅ v11 export complete:", OUTPUT_XLSX)
display(acc_dept.head(10) if acc_dept is not None else None)
display(staff_m.head(10))

17:00:13 - cmdstanpy - INFO - Chain [1] start processing
17:00:13 - cmdstanpy - INFO - Chain [1] done processing
17:00:13 - cmdstanpy - INFO - Chain [1] start processing
17:00:23 - cmdstanpy - INFO - Chain [1] done processing
17:00:24 - cmdstanpy - INFO - Chain [1] start processing
17:00:35 - cmdstanpy - INFO - Chain [1] done processing
17:00:36 - cmdstanpy - INFO - Chain [1] start processing
17:00:48 - cmdstanpy - INFO - Chain [1] done processing
17:00:49 - cmdstanpy - INFO - Chain [1] start processing
17:00:49 - cmdstanpy - INFO - Chain [1] done processing
17:00:50 - cmdstanpy - INFO - Chain [1] start processing
17:01:02 - cmdstanpy - INFO - Chain [1] done processing
17:01:03 - cmdstanpy - INFO - Chain [1] start processing
17:01:14 - cmdstanpy - INFO - Chain [1] done processing
17:01:15 - cmdstanpy - INFO - Chain [1] start processing
17:01:27 - cmdstanpy - INFO - Chain [1] done processing
17:01:28 - cmdstanpy - INFO - Chain [1] start processing
17:01:28 - cmdstanpy - INFO - Chain [1]

KeyError: 'vertical'